In [1]:
%%time

from adlfs import AzureBlobFileSystem

import time

import json
import pandas
import pandas as pd
from tqdm import tqdm

from common.schemas.pyarrow_schema import tagging_schema
from common.storage.azure_file_storage import AzureFileStorageAdapter
from common.captioning.azure_descriptions import AzureCaption
from common.schemas.pyarrow_schema import schema
from PIL import Image
import requests


tqdm.pandas(desc="Progress")
file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

from common.functions.functions import Functions

functions: Functions = Functions()

caption: AzureCaption = AzureCaption(file_system)

CPU times: total: 1.25 s
Wall time: 9.33 s


In [2]:
%%time

curated_df = pandas.read_parquet('data/parquet/back.parquet', filesystem=file_system, engine='pyarrow')

CPU times: total: 234 ms
Wall time: 4.68 s


In [3]:
%%time

captions = pd.read_parquet('data/parquet/image_captions.parquet', filesystem=file_system, engine='pyarrow')
tags = pd.read_parquet('data/parquet/image_tags.parquet', filesystem=file_system, engine='pyarrow')
crops = pd.read_parquet('data/parquet/image_cropping.parquet', filesystem=file_system, engine='pyarrow')

display(captions.shape)
display(tags.shape)
display(crops.shape)

(31506, 7)

(71716, 3)

(3545, 6)

CPU times: total: 46.9 ms
Wall time: 2.38 s


In [4]:
%%time

current_captions = file_system.ls("data/caption")
display(len(current_captions))

3568

CPU times: total: 469 ms
Wall time: 5.1 s


In [5]:
%%time

all_data = []
for caption_file in tqdm(current_captions, total=len(current_captions), desc='Reading caption files'):
	caption_data = json.loads(file_system.read_text(caption_file, encoding='utf-8'))
	dense_caption_result = caption_data.get('denseCaptionsResult')
	metadata = caption_data.get('metadata')
	tags_result = caption_data.get('tagsResult')
	smart_crop_result = caption_data.get('smartCropsResult')
	basic_caption = caption_data.get('captionResult')
	image_id = caption_file.split('/')[-1].split('.')[0]
	filtered_data = {
		"id": image_id,
		'captions': [basic_caption],
		"dense_captions": dense_caption_result['values'],
		"meta": [metadata],
		"tags": tags_result['values'],
		"smart_crop": smart_crop_result['values']
	}
	all_data.append(filtered_data)

Reading caption files: 100%|██████████| 3568/3568 [06:49<00:00,  8.72it/s]  


CPU times: total: 27.2 s
Wall time: 6min 49s


In [6]:
%%time

new_captions = pandas.json_normalize(data=all_data, record_path=['dense_captions'], meta=['id'], record_prefix='dense_captions_')
new_tags = pandas.json_normalize(data=all_data, record_path=['tags'], meta=['id'], record_prefix='tags_')
new_crops = pandas.json_normalize(data=all_data, record_path=['smart_crop'], meta=['id'], record_prefix='smart_crop_')

CPU times: total: 1.03 s
Wall time: 3.11 s


In [7]:
%%time

new_basic_captions = pandas.json_normalize(data=all_data, record_path=['captions'], meta=['id'], record_prefix='captions_')
meta = pandas.json_normalize(data=all_data, record_path=['meta'], meta=['id'], record_prefix='meta_')

CPU times: total: 0 ns
Wall time: 78.1 ms


In [8]:
%%time

display(new_captions)
display(new_tags)
display(new_crops)
display(new_basic_captions)
display(meta)

,dense_captions_text,dense_captions_confidence,dense_captions_boundingBox.x,dense_captions_boundingBox.y,dense_captions_boundingBox.w,dense_captions_boundingBox.h,id
0,a woman taking a selfie,0.662057,0,0,1468,2608,1002cx2
1,a woman taking a selfie,0.665344,0,100,1447,2473,1002cx2
2,a woman wearing a black dress,0.473444,185,1443,1265,1150,1002cx2
3,a close up of a key,0.593095,716,1503,82,140,1002cx2
4,a close up of an eye,0.619898,778,744,152,84,1002cx2
...,...,...,...,...,...,...,...
31728,a close-up of a green plant,0.453059,680,1066,394,280,zzxq2l
31729,a mountain range in the distance,0.334624,498,277,568,99,zzxq2l
31730,a woman smiling at the camera,0.465720,244,293,319,530,zzxq2l
31731,a woman wearing a dress,0.548892,0,758,1048,570,zzxq2l


,tags_name,tags_confidence,id
0,person,0.998358,1002cx2
1,human face,0.997763,1002cx2
2,clothing,0.990993,1002cx2
3,lady,0.981919,1002cx2
4,smile,0.964994,1002cx2
...,...,...,...
72068,woman,0.753262,zzxq2l
72069,beach,0.739799,zzxq2l
72070,standing,0.630927,zzxq2l
72071,girl,0.556561,zzxq2l


,smart_crop_aspectRatio,smart_crop_boundingBox.x,smart_crop_boundingBox.y,smart_crop_boundingBox.w,smart_crop_boundingBox.h,id
0,1.0,0,140,1462,1462,1002cx2
1,1.0,143,138,1249,1250,1003dod
2,1.0,206,0,3013,3012,10044wv
3,1.0,67,10,397,396,1005neb
4,1.0,0,253,1025,1025,1007zxc
...,...,...,...,...,...,...
3563,1.0,0,0,1025,1025,zzr9nz
3564,1.0,0,0,461,461,zzruf4
3565,1.0,0,0,2273,2273,zzu8i8
3566,1.0,135,0,1299,1299,zzvv3x


,captions_text,captions_confidence,id
0,a woman taking a selfie,0.662057,1002cx2
1,a woman in a garment standing next to a tree,0.513081,1003dod
2,a train on a bridge over a body of water,0.382730,10044wv
3,an aerial view of a city at night,0.567479,1005neb
4,fireworks above a city,0.507308,1007zxc
...,...,...,...
3563,two women in garments on a beach,0.529292,zzr9nz
3564,a woman with tattoos on her arm,0.501798,zzruf4
3565,a woman in a blue lingerie,0.430763,zzu8i8
3566,a woman sitting on a bed,0.584322,zzvv3x


,meta_width,meta_height,id
0,1468,2608,1002cx2
1,1523,2030,1003dod
2,4032,3024,10044wv
3,612,408,1005neb
4,1080,1350,1007zxc
...,...,...,...
3563,1080,1350,zzr9nz
3564,570,704,zzruf4
3565,2309,2560,zzu8i8
3566,1440,1800,zzvv3x


CPU times: total: 15.6 ms
Wall time: 109 ms


In [9]:
merge_singles = pandas.merge(new_basic_captions, meta, on='id').set_index(keys=['id'], drop=False)
merge_singles.drop_duplicates(inplace=True)
merge_singles.reset_index(drop=True, inplace=True)
display(merge_singles)

,captions_text,captions_confidence,id,meta_width,meta_height
0,a woman taking a selfie,0.662057,1002cx2,1468,2608
1,a woman in a garment standing next to a tree,0.513081,1003dod,1523,2030
2,a train on a bridge over a body of water,0.382730,10044wv,4032,3024
3,an aerial view of a city at night,0.567479,1005neb,612,408
4,fireworks above a city,0.507308,1007zxc,1080,1350
...,...,...,...,...,...
3563,two women in garments on a beach,0.529292,zzr9nz,1080,1350
3564,a woman with tattoos on her arm,0.501798,zzruf4,570,704
3565,a woman in a blue lingerie,0.430763,zzu8i8,2309,2560
3566,a woman sitting on a bed,0.584322,zzvv3x,1440,1800


In [10]:
merged_to_curate = pandas.merge(merge_singles, curated_df, on='id', how='outer').set_index(keys=['id'], drop=False)
merged_to_curate.fillna(value='', inplace=True)
display(merged_to_curate)

,captions_text,captions_confidence,id,meta_width,meta_height,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,,,,,
1002cx2,a woman taking a selfie,0.662057,1002cx2,1468.0,2608.0,SFWRedheads,Sayleywayley,Happy New Year to every single person near and...,blonde woman with red hair and black bra top p...,1fb785ca16b10f4f7613961f0b88f369,/r/SFWRedheads/comments/1002cx2/happy_new_year...,https://i.redd.it/el3s490lzb9a1.jpg,1002cx2.jpg,data/image/1002cx2.jpg,RedHeadDiffusion,True,True,True,[]
1003dod,a woman in a garment standing next to a tree,0.513081,1003dod,1523.0,2030.0,HotGirlNextDoor,fairytale808,pink and blonde (iktr),blonde woman in pink bikinisuit standing on a ...,0849c7aafe126ad27b58cb6e6c9c6592,/r/HotGirlNextDoor/comments/1003dod/pink_and_b...,https://i.redd.it/x56eekrd8c9a1.jpg,1003dod.jpg,data/image/1003dod.jpg,SexyDiffusion,True,True,True,[]
10044wv,a train on a bridge over a body of water,0.38273,10044wv,4032.0,3024.0,CityPorn,dawginphilly,"Wissahickon, Phila, PA",arafed view of a train on a bridge over a river,abb87d7618eec7cd6a86d7647263ea1b,/r/CityPorn/comments/10044wv/wissahickon_phila...,https://i.redd.it/4y6hzocyec9a1.jpg,10044wv.jpg,data/image/10044wv.jpg,CityDiffusion,True,True,True,[]
1005neb,an aerial view of a city at night,0.567479,1005neb,612.0,408.0,CityPorn,drizzydriller,"Madrid, Spain: You have to appreciate the layo...",a view of a city at night with a lot of lights...,252c222cce39813a08878117b63723d9,/r/CityPorn/comments/1005neb/madrid_spain_you_...,https://i.redd.it/o4sh9b7csc9a1.jpg,1005neb.jpg,data/image/1005neb.jpg,CityDiffusion,True,True,True,[]
1007zxc,fireworks above a city,0.507308,1007zxc,1080.0,1350.0,CityPorn,Poohbizzle79,"Berlin, Germany.",fireworks are lit up in the night sky above a ...,8a85fb4a9347fa86c080d2f178663ccc,/r/CityPorn/comments/1007zxc/berlin_germany/,https://i.redd.it/06iww0mced9a1.jpg,1007zxc.jpg,data/image/1007zxc.jpg,CityDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,,,13d2hsf,,,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,someone holding a cup of ice cream in their hand,c6788074138175c661fc652bd7630e33,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,True,True,[]
13d661b,,,13d661b,,,bathandbodyworks,xeloux,Facebook market finds!,three candles are sitting on a blanket on a bed,e868289445c195815dd67dd0df1a65cb,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,True,True,[]
13d8s0i,,,13d8s0i,,,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,someone holding a tube of body cream in a store,965bc5a58a597192700234729758101b,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,True,False,[]


In [11]:
%%time

merged_captions = pandas.concat([new_captions, captions])
merged_captions.set_index(keys=['id', 'dense_captions_text', 'dense_captions_confidence'], inplace=True, drop=False)
merged_captions.drop_duplicates(inplace=True)
merged_captions.reset_index(drop=True, inplace=True)

display(f'{merged_captions.shape[0] - captions.shape[0]} new rows added to captions')

merged_captions.to_parquet('data/parquet/image_captions.parquet', filesystem=file_system, engine='pyarrow')
display(pandas.read_parquet('data/parquet/image_captions.parquet', filesystem=file_system, engine='pyarrow'))

'227 new rows added to captions'

,dense_captions_text,dense_captions_confidence,dense_captions_boundingBox.x,dense_captions_boundingBox.y,dense_captions_boundingBox.w,dense_captions_boundingBox.h,id
0,a woman taking a selfie,0.662057,0,0,1468,2608,1002cx2
1,a woman taking a selfie,0.665344,0,100,1447,2473,1002cx2
2,a woman wearing a black dress,0.473444,185,1443,1265,1150,1002cx2
3,a close up of a key,0.593095,716,1503,82,140,1002cx2
4,a close up of an eye,0.619898,778,744,152,84,1002cx2
...,...,...,...,...,...,...,...
31728,a close-up of a green plant,0.453059,680,1066,394,280,zzxq2l
31729,a mountain range in the distance,0.334624,498,277,568,99,zzxq2l
31730,a woman smiling at the camera,0.465720,244,293,319,530,zzxq2l
31731,a woman wearing a dress,0.548892,0,758,1048,570,zzxq2l


CPU times: total: 297 ms
Wall time: 6.68 s


In [12]:
%%time

merged_tags = pandas.concat([new_tags, tags])
merged_tags.set_index(keys=['id', 'tags_name', 'tags_confidence'], inplace=True, drop=False)
merged_tags.drop_duplicates(inplace=True)
merged_tags.reset_index(drop=True, inplace=True)

display(f'{merged_tags.shape[0] - tags.shape[0]} new rows added to tags')

merged_tags.to_parquet('data/parquet/image_tags.parquet', filesystem=file_system, engine='pyarrow')
display(pandas.read_parquet('data/parquet/image_tags.parquet', filesystem=file_system, engine='pyarrow'))

'357 new rows added to tags'

,tags_name,tags_confidence,id
0,person,0.998358,1002cx2
1,human face,0.997763,1002cx2
2,clothing,0.990993,1002cx2
3,lady,0.981919,1002cx2
4,smile,0.964994,1002cx2
...,...,...,...
72068,woman,0.753262,zzxq2l
72069,beach,0.739799,zzxq2l
72070,standing,0.630927,zzxq2l
72071,girl,0.556561,zzxq2l


CPU times: total: 141 ms
Wall time: 2.64 s


In [13]:
%%time

merged_crops = pandas.concat([new_crops, crops])
merged_crops.set_index(keys=['id'], inplace=True, drop=False)
merged_crops.drop_duplicates(inplace=True)
merged_crops.reset_index(drop=True, inplace=True)

display(f'{merged_crops.shape[0] - crops.shape[0]} new rows added to crops')

merged_crops.to_parquet('data/parquet/image_cropping.parquet', filesystem=file_system, engine='pyarrow')

display(pandas.read_parquet('data/parquet/image_cropping.parquet', filesystem=file_system, engine='pyarrow'))

'23 new rows added to crops'

,smart_crop_aspectRatio,smart_crop_boundingBox.x,smart_crop_boundingBox.y,smart_crop_boundingBox.w,smart_crop_boundingBox.h,id
0,1.0,0,140,1462,1462,1002cx2
1,1.0,143,138,1249,1250,1003dod
2,1.0,206,0,3013,3012,10044wv
3,1.0,67,10,397,396,1005neb
4,1.0,0,253,1025,1025,1007zxc
...,...,...,...,...,...,...
3563,1.0,0,0,1025,1025,zzr9nz
3564,1.0,0,0,461,461,zzruf4
3565,1.0,0,0,2273,2273,zzu8i8
3566,1.0,135,0,1299,1299,zzvv3x


CPU times: total: 109 ms
Wall time: 2.99 s


In [14]:
import os
def create_thumbnail(image_id_, curated_df_, crops_, extant_files_):
	try:
		record = None
		cropping_information = None
		if image_id_ in extant_files_:
			return None
		try:
			record = curated_df.loc[curated_df['id'] == image_id_]
			cropping_information = crops.loc[crops['id'] == image_id_]
		except KeyError or IndexError:
			return None
		if record is None and cropping_information is None and len(record) == 0 and len(cropping_information) == 0:
			return None
		if record is not None and cropping_information is not None:
			record = curated_df_.loc[curated_df_['id'] == image_id_]
			cropping_information = crops_.loc[crops['id'] == image_id_]
			try:
				image_url = file_system.url(record.path.values[0])
			except Exception as e:
				display(f'Error creating thumbnail for {image_id_}: {e}', clear=True)
				return None
			original_image = Image.open(requests.get(image_url, stream=True).raw)
			copied_image = original_image.copy()
			original_image.close()
			try:
				cropped = copied_image.crop((cropping_information['smart_crop_boundingBox.x'].values[0], cropping_information['smart_crop_boundingBox.y'].values[0], cropping_information['smart_crop_boundingBox.x'].values[0] + cropping_information['smart_crop_boundingBox.w'].values[0], cropping_information['smart_crop_boundingBox.y'].values[0] + cropping_information['smart_crop_boundingBox.h'].values[0]))
			except Exception as e:
				display(f'Error creating thumbnail for {image_id_}: {e}', clear=True)
				return None
			copied_image.close()
			resized = cropped.resize((512, 512), 1)
			cropped.close()
			resized.save('temp.jpg')
			file_system.upload('temp.jpg', f'data/image/thumbnail/{image_id_}.jpg', overwrite=True)
			display(f'Thumbnail created for {image_id_}', clear=True)
			return None
		else:
			return None
	except Exception as e:
		display(f'Error creating thumbnail for {image_id_}: {e}', clear=True)
		return None

In [15]:
%%time

extant = [os.path.basename(item.replace('\n', '').split('.')[0]) for item in file_system.ls('data/image/thumbnail')]
display(extant)

['1002cx2',
 '1003dod',
 '1008ddt',
 '100dcrd',
 '100jl1b',
 '100km4t',
 '100knfl',
 '100lihz',
 '100nibu',
 '100nndx',
 '100qk25',
 '100s0s5',
 '100sez9',
 '100vc0h',
 '100xqd2',
 '1011gjo',
 '10133o2',
 '1013bdt',
 '1013fun',
 '1019kyo',
 '101w7lh',
 '1027i9a',
 '102yo07',
 '1032j8y',
 '1035nya',
 '103h4hd',
 '103nm31',
 '103zqoc',
 '104it0i',
 '105dxeb',
 '105lrts',
 '105mekt',
 '105qvgl',
 '105rl3w',
 '105rpcj',
 '105spve',
 '105styc',
 '105tw6t',
 '105v03t',
 '105xfdg',
 '105zktf',
 '10623sj',
 '1062601',
 '106673i',
 '1067bcu',
 '1067vw4',
 '1068gp7',
 '1069pzt',
 '106dg98',
 '106hchf',
 '106lsw4',
 '106mh03',
 '106ohb7',
 '106p92p',
 '106quap',
 '106xkyj',
 '106yv2j',
 '106yxk3',
 '106z40r',
 '1071ykc',
 '1072139',
 '1072gmf',
 '1072vdp',
 '10741ga',
 '107gd1b',
 '107x4qv',
 '108e7ml',
 '1096qs1',
 '109n66u',
 '109pem3',
 '109wqn0',
 '109x870',
 '10a3qnh',
 '10a5w3r',
 '10a6m9y',
 '10a928m',
 '10aajfj',
 '10ab2dk',
 '10agu6e',
 '10aijhh',
 '10alnrk',
 '10aol8b',
 '10ap6iv',
 '10

CPU times: total: 1.11 s
Wall time: 8.93 s


In [ ]:
%%time

for elem in tqdm(curated_df.id.values, total=len(curated_df.id.values), desc='Creating thumbnails'):
	create_thumbnail(elem, curated_df, crops, extant)

"Error creating thumbnail for 100dbjr: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))"

Creating thumbnails:   2%|▏         | 860/37031 [3:19:20<34479:13:07, 3431.62s/it]

In [ ]:
!jupyter notebook stop